# Exploratory Data Analysis 

## import libraries 

In [152]:
import geopandas as gpd
from pygeos import STRtree
from pygeos.creation import points, polygons
from pygeos.geometry import Geometry
from time import time

## GBIF EDA



## load data 


In [21]:
# load gbif 
gbif = gpd.read_file('../data/clean_data/vancouver_gbif.shp')

# load parks 
parks = gpd.read_file('../data/clean_data/vancouver_parks.shp')

In [3]:
gbif.head()

,kingdom,phylum,class,order,family,genus,species,latitude,longitude,timestamp,day,month,year,basis_of_r,geometry
0,Fungi,Basidiomycota,Dacrymycetes,Dacrymycetales,Dacrymycetaceae,Dacrymyces,Dacrymyces chrysospermus,49.305487,-123.138538,2015-02-08T13:15:50Z,8,2,2015,HUMAN_OBSERVATION,POINT (-123.13854 49.30549)
1,Animalia,Chordata,Aves,Anseriformes,Anatidae,Branta,Branta canadensis,49.279785,-123.138956,2015-02-08T16:28:30Z,8,2,2015,HUMAN_OBSERVATION,POINT (-123.13896 49.27978)
2,Animalia,Chordata,Aves,Anseriformes,Anatidae,Anas,Anas platyrhynchos,49.276400,-123.145512,2015-02-09T08:50:46Z,9,2,2015,HUMAN_OBSERVATION,POINT (-123.14551 49.27640)
3,Animalia,Chordata,Aves,Anseriformes,Anatidae,Lophodytes,Lophodytes cucullatus,49.279637,-123.139337,2015-02-09T08:42:00Z,9,2,2015,HUMAN_OBSERVATION,POINT (-123.13934 49.27964)
4,Animalia,Chordata,Aves,Passeriformes,Emberizidae,Melospiza,Melospiza melodia,49.285406,-123.143340,2015-02-09T09:51:57Z,9,2,2015,HUMAN_OBSERVATION,POINT (-123.14334 49.28541)


In [19]:
print("The count of total observations: {}.\n".format(gbif.shape[0]))
print("The years of observations span {0} to {1}.\n".format(gbif.year.min(), gbif.year.max()))
print("The count of unique months where observations are made: {}.\n".format(gbif.month.nunique()))

The count of total observations: 5110.

The years of observations span 2009 to 2019.

The count of unique months where observations are made: 12.



In [18]:
taxonomy = ['kingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species']
for level in taxonomy:
    print("The count of unique {0} represented is: {1}\n".format(level, gbif[level].nunique()))

The count of unique kingdom represented is: 5

The count of unique phylum represented is: 16

The count of unique class represented is: 43

The count of unique order represented is: 135

The count of unique family represented is: 350

The count of unique genus represented is: 672

The count of unique species represented is: 881



# get species that are observed in parks 

In order to capture species observations that occurr right along the border of parks, each park polygon will be buffered to 10m so that observations that are on or just outside the official park border are included. 

### buffer parks 

Will do this later when I find the correct CRS to re-project the GBIF data into. 

In [23]:
# clip the gbif data to inside parks 
clipped_gbif = gpd.clip(gdf=gbif, mask=parks)

In [24]:
print("The count of park observations: {}.\n".format(clipped_gbif.shape[0]))
print("The years of parks observations span {0} to {1}.\n".format(gbif.year.min(), clipped_gbif.year.max()))
print("The count of unique months where park observations are made: {}.\n".format(clipped_gbif.month.nunique()))

The count of park observations: 2638.

The years of parks observations span 2009 to 2019.

The count of unique months where park observations are made: 12.



In [25]:
taxonomy = ['kingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species']
for level in taxonomy:
    print("The count of unique {0} represented is: {1}\n".format(level, clipped_gbif[level].nunique()))

The count of unique kingdom represented is: 5

The count of unique phylum represented is: 15

The count of unique class represented is: 37

The count of unique order represented is: 107

The count of unique family represented is: 244

The count of unique genus represented is: 429

The count of unique species represented is: 556



In [41]:
# proportion of gbif that is found in parks 
prop = round((len(clipped_gbif)/len(gbif))*100, 2)
print("Percentage of gbif obsevations that are found in parks: {} %.".format(prop))

Percentage of gbif obsevations that are found in parks: 51.62 %.


## Parks EDA


In [26]:
parks.head()

,park_name,park_type,park_prima,latitude,longitude,geometry
0,Locarno Park,Local,Park,49.274275,-123.207753,"MULTIPOLYGON (((-123.21241 49.27645, -123.2120..."
1,Macdonald Park,Local,Park,49.229759,-123.098392,"POLYGON ((-123.09928 49.23014, -123.09747 49.2..."
2,Maclean Park,Local,Park,49.278819,-123.088448,"POLYGON ((-123.08750 49.27845, -123.08757 49.2..."
3,Major Matthews Park,Local,Park,49.261583,-123.107233,"POLYGON ((-123.10744 49.26170, -123.10702 49.2..."
4,Malkin Park,Local,Park,49.231957,-123.176939,"POLYGON ((-123.17791 49.23119, -123.17791 49.2..."


In [32]:
print("The count of total parks: {}\n".format(parks.park_name.nunique()))
print("The count of unique park types: {}\n".format(parks.park_type.nunique()))
print("The count of unique primary park uses: {}\n".format(parks.park_prima.nunique()))

The count of total parks: 227

The count of unique park types: 2

The count of unique primary park uses: 9



In [35]:
for park_type in parks.park_type.unique():
    print(park_type)

Local
Regional


In [36]:
for use in parks.park_prima.unique():
    print(use)

Park
Green Space
Trail
Golf Course
Athletic
Playground
Plaza
Water Access
School Park


# determine species count for each park 

### build STR tree

In [72]:
coords = points(clipped_gbif.longitude, clipped_gbif.latitude)
gbif_tree = STRtree(coords)

In [73]:
pacific_spirit = parks.query('park_name == "Pacific Spirit Park"')
pacific_spirit

,park_name,park_type,park_prima,latitude,longitude,geometry
225,Pacific Spirit Park,Regional,Park,49.251873,-123.19527,"POLYGON ((-123.19604 49.25463, -123.19604 49.2..."


In [145]:
polygons(Geometry(str(pacific_spirit.geometry.values[0])))

GEOSException: Shell is not a LinearRing

In [140]:
pac_spirit_geom = Geometry(str(pacific_spirit.geometry.values[0]))

In [141]:
gbif_tree.query(pac_spirit_geom, predicate='contains')

SystemError: <method 'query' of 'pygeos.lib.STRtree' objects> returned a result with an error set

In [82]:
type(pac_spirit_geom)

pygeos.lib.GEOSGeometry

# use geopandas clip instead 

In [155]:
truncated_parks = parks.query('park_name == "Pacific Spirit Park" | park_name == "Stanley Park"')
truncated_parks

,park_name,park_type,park_prima,latitude,longitude,geometry
150,Stanley Park,Local,Park,49.302136,-123.140049,"POLYGON ((-123.13845 49.29360, -123.14156 49.2..."
225,Pacific Spirit Park,Regional,Park,49.251873,-123.195270,"POLYGON ((-123.19604 49.25463, -123.19604 49.2..."


In [177]:
type(truncated_parks)

geopandas.geodataframe.GeoDataFrame

In [171]:
num_species = []
for park in truncated_parks.geometry:
    start_time = time()
    
    # clip gbif points to a single park 
    park_species = gpd.clip(gdf=gbif, mask=park)
    
    stop_time = time()
    total = stop_time - start_time
    print("Clipping time for 1 park: {}".format(total))
    
    # store the number of species clipped for each park 
    num_species.append(len(park_species))
    

Clipping time for 1 park: 0.8060381412506104
Clipping time for 1 park: 0.3615880012512207


In [180]:
def add_species_counts(species_gdf, parks_gdf):
    """
    Add a column to parks_gdf for count of species that have been clipped to inside the park. 
    
    Arguments
    ---------
    species_gdf (GeoDataFrame) : a dataframe of species occurrence data from GBIF. 
    parks_gdf  (GeoDataFrame)  : a dataframe of parks polygons. 
    
    Returns 
    -------
    GeoDataFrame
        Produces the parks_gdf with an added column of species counts for each park polygon. 
        
    Examples
    --------
    TODO 
    
    """
    parks = parks_gdf.copy()
    
    # apply clipping to each park and get species count per park 
    num_species = parks.geometry.apply(lambda park: len(gpd.clip(gdf=species_gdf, mask=park)))
    
    # add species count list to parks_gdf
    parks['species_count'] = num_species
    
    return parks

In [181]:
add_species_counts(species_gdf=clipped_gbif, parks_gdf=truncated_parks)

,park_name,park_type,park_prima,latitude,longitude,geometry,species_count
150,Stanley Park,Local,Park,49.302136,-123.140049,"POLYGON ((-123.13845 49.29360, -123.14156 49.2...",1422
225,Pacific Spirit Park,Regional,Park,49.251873,-123.195270,"POLYGON ((-123.19604 49.25463, -123.19604 49.2...",9
